## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2021/seasontype/2

In [1]:
SET_WEEK = 8
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_8


In [2]:
from scrapers.scrape_2021 import get_2021_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2021 data

In [4]:
%%time

X = get_2021_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game', 'offense_passing_AVG', 'offense_passing_YDS/G', 'offense_passing_RTG', 
             'offense_receiving_AVG', 'offense_rushing_YDS/G', 'defense_points_per_game', 'defense_passing_AVG', 
             'defense_receiving_AVG', 'defense_rushing_YDS/G', 'defense_passing_SYL_per_game', 
             'offense_downs_Third Downs_ATT_per_game', 'defense_downs_Third Downs_ATT_per_game', 
             'defense_downs_Fourth Downs_ATT_per_game', 'defense_pass_to_rush_ratio', 
             'defense_downs_First Downs_penalty_ratio', 'offense_passing_TD_per_game', 'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 21.8 s


,Team,games_played,season,offense_points_per_game,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,offense_rushing_YDS/G,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_Fourth Downs_ATT_per_game,defense_pass_to_rush_ratio,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,offense_pass_TD_to_INT
0,Dallas Cowboys,7,2021,32.1,8.5,302.9,112.1,11.9,152.0,23.1,...,11.9,88.3,14.857143,12.714286,11.857143,1.571429,1.827586,0.130435,2.571429,3.6
1,Tampa Bay Buccaneers,8,2021,32.5,7.7,327.5,108.3,11.5,95.6,22.9,...,9.8,78.0,11.625000,13.000000,12.250000,1.125000,2.025157,0.131737,3.125000,5.0


In [5]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [6]:
X['games_played'].unique().tolist()

[7, 8]

In [14]:
# X[X['games_played'] == 6]

In [8]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 4

predictions.sort_values('pred_mean', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
7,Arizona Cardinals,2021,week_8,0.293435,0.265594,0.437662,0.377454,0.343536
4,Los Angeles Rams,2021,week_8,0.325420,0.329189,0.277161,0.341629,0.318350
5,Buffalo Bills,2021,week_8,0.287594,0.248868,0.323894,0.374347,0.308676


In [9]:
predictions.to_csv('weekly_runs/' + week_name + '.csv', index = False)

### Check weekly run

In [10]:
predictions

,Team,season,week_marker,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean
7,Arizona Cardinals,2021,week_8,0.293435,0.265594,0.437662,0.377454,0.343536
4,Los Angeles Rams,2021,week_8,0.325420,0.329189,0.277161,0.341629,0.318350
5,Buffalo Bills,2021,week_8,0.287594,0.248868,0.323894,0.374347,0.308676
0,Dallas Cowboys,2021,week_8,0.275172,0.228097,0.348953,0.287019,0.284810
1,Tampa Bay Buccaneers,2021,week_8,0.259118,0.247275,0.332884,0.295580,0.283714
8,Las Vegas Raiders,2021,week_8,0.218720,0.175781,0.284814,0.254564,0.233470
13,Cincinnati Bengals,2021,week_8,0.189056,0.094028,0.209979,0.251121,0.186046
6,Minnesota Vikings,2021,week_8,0.110206,0.064635,0.119675,0.194733,0.122312
2,Baltimore Ravens,2021,week_8,0.121537,0.058621,0.110808,0.188005,0.119743
12,San Francisco 49ers,2021,week_8,0.097981,0.063474,0.140095,0.177329,0.119720
